In [74]:
from islpy import *

# Python bindings for the integer set library (isl)

This notebook illustrates some of the core functionality available in the integer set library (isl). You should refer to the corresponding documentation here:  
[https://documen.tician.de/islpy/](https://documen.tician.de/islpy/)

The python bindings are effectively a python wrapper around the underlying isl library:  
[https://libisl.sourceforge.io/manual.pdf](https://libisl.sourceforge.io/manual.pdf)

The two most important objects in isl are sets and maps and there are three flavors of each.

# Sets

There are three kinds of set objects in isl:
* [`BasicSet`](https://documen.tician.de/islpy/ref_set.html#islpy.BasicSet): convex domains
* [`Set`](https://documen.tician.de/islpy/ref_set.html#set): disjoint union of basic sets
* [`UnionSet`](https://documen.tician.de/islpy/ref_set.html#union-set): disjoint union of sets


Sets are defined by lists of constraints involving one or more dimensions. Dimensions are reference using `dim_type` objects:  
[https://documen.tician.de/islpy/reference.html#symbolic-constants](https://documen.tician.de/islpy/reference.html#symbolic-constants)


## Basic sets

Consider the following convex 3D domain defining the $N$ by $N$ by $N$ cube in $\mathbb{Z}^3$. In this example, there is 1 parameter constraint and 3 output constraints. Note that `dim_type.in_` don't have any meaning in set objects. These are used in maps below.

In [75]:
D = BasicSet('[N]->{[i,j,k] : 0<=i,j,k<=N}')

nb_params = D.dim(dim_type.param)
nb_in = D.dim(dim_type.in_)
nb_out = D.dim(dim_type.out)
nb_cst = D.dim(dim_type.cst)

print('num params: {}'.format(nb_params))
print('num inputs: {}'.format(nb_in))
print('num outputs: {}'.format(nb_out))
print('num constants: {}'.format(nb_cst))

num params: 1
num inputs: 0
num outputs: 3
num constants: 1


Note that there are six constraints. This corresponds to the fact that a cube has six faces.

In [76]:
print('constraints:')
for c in D.get_constraints():
    print(c)

constraints:
[N] -> { [i, j, k] : i >= 0 }
[N] -> { [i, j, k] : j >= 0 }
[N] -> { [i, j, k] : k >= 0 }
[N] -> { [i, j, k] : N - i >= 0 }
[N] -> { [i, j, k] : N - j >= 0 }
[N] -> { [i, j, k] : N - k >= 0 }


Under the hood, the constraints are representated by two `Mat` (matrix) objects. One for the **equality** constraints and one for the **inequality** constraints.

Each matrix contains one row per constraint, and each column represents each of the parameters, indices, and constant terms. In the example above, each matrix has 1+3+1 = 5 columns (for the "N" + "i,j,k" + constant terms). There are no equality constraints. `BasicSet` objects have functions to get the corresponding matrices, but you have specify the dimensions you want.

In [77]:
ineq_matrix = D.inequalities_matrix(dim_type.param, dim_type.in_, dim_type.out, dim_type.cst)

There is no out-of-box function to convert `Mat` objects to python primitive arrays. But we can define a simple helper function for this. In this example, the constraint $i \ge 0$ is represented in the inequality constraint matrix with the row:  

`[0,1,0,0,0]`

where as the constraint $N-i \ge 0$ is represented by the row:  

`[1,-1,0,0,0]`

In [78]:
# isl Mat elements are Val objects
# isl Val objects represent rational numbers (i.e., ratios of integers), when the denominator is 1
# the "to_python" function converts it to a long data type
def to_array(matrix, as_long=False):
    nb_rows = matrix.rows()
    nb_cols = matrix.cols()
    return [[matrix.get_element_val(i,j).to_python() if as_long else matrix.get_element_val(i,j) for j in range(nb_cols)] for i in range(nb_rows)]

def pretty_print(array):
    print('[')
    [print('  {}'.format(row)) for row in array]
    print(']')

val_array = to_array(ineq_matrix)
long_array = to_array(ineq_matrix, as_long=True)

print('constraints array (as Val objects)')
pretty_print(val_array)

print('constraints array (as longs)')
pretty_print(long_array)

constraints array (as Val objects)
[
  [Val("0"), Val("1"), Val("0"), Val("0"), Val("0")]
  [Val("0"), Val("0"), Val("1"), Val("0"), Val("0")]
  [Val("0"), Val("0"), Val("0"), Val("1"), Val("0")]
  [Val("1"), Val("-1"), Val("0"), Val("0"), Val("0")]
  [Val("1"), Val("0"), Val("-1"), Val("0"), Val("0")]
  [Val("1"), Val("0"), Val("0"), Val("-1"), Val("0")]
]
constraints array (as longs)
[
  [0, 1, 0, 0, 0]
  [0, 0, 1, 0, 0]
  [0, 0, 0, 1, 0]
  [1, -1, 0, 0, 0]
  [1, 0, -1, 0, 0]
  [1, 0, 0, -1, 0]
]


## Sets

Consider the following non-convex 3D domain defining the two $N$ by $N$ by $N$ cubes in $\mathbb{Z}^3$.

In [79]:
D = Set('[N]->{[i,j,k] : 0<=i,j,k<N or N<=i,j,k<2N}')

This set has two basic sets:

In [80]:
print(D.n_basic_set())

2


In [81]:
for bs in D.get_basic_sets():
    print(bs)

[N] -> { [i, j, k] : 0 <= i < N and 0 <= j < N and 0 <= k < N }
[N] -> { [i, j, k] : N <= i < 2N and N <= j < 2N and N <= k < 2N }


Many of the operations on `BasicSet`s are also available on `Set`s. Notice that each `BasicSet` in a `Set` is the same "type" meaning they all have the same number of parameters and indices.

The domains charaterized by `Set` objects can be "named" with `*_tuple_*` functions:

In [82]:
named_D = D.set_tuple_name('A')
print(named_D)

[N] -> { A[i, j, k] : (0 <= i < N and 0 <= j < N and 0 <= k < N) or (N <= i < 2N and N <= j < 2N and N <= k < 2N) }


## UnionSets

The union of a 2D basic set and a 3D basic set is not well defined by `Set` objects. To hold sets of different dimensionalities, `UnionSet`s are used.

For example, consider the following union set which represents the 2D domain on the $ij$-plane denoted by the named set "A" and the 3D domain in the $ijk$-space denoted by the named set "B". These `UnionSet` objects are useful for representing program statement instances. Think of "A" and "B" as two statements appearing in some triply nested loop. We will think of this from a code generation perspective below.

For now, just note that `UnionSet` represent unions of named `Set` objects.

In [83]:
D = UnionSet('[N]->{A[i,j]: 0<=i,j<N; B[i,j,k] : 0<=i,j,k<N}')

# Maps

Before we can talk about code generation, we must talk about maps. Like sets, there are three kinds of map objects in isl:
* [`BasicMap`](https://documen.tician.de/islpy/ref_set.html#basic-map): functions between convex domains
* [`Map`](https://documen.tician.de/islpy/ref_set.html#map): functions between unions of similar convex domains
* [`UnionMap`](https://documen.tician.de/islpy/ref_set.html#union-map): functions between unions of (dis)similar convex domains


Like sets, maps are defined over inputs (domains) and outputs (ranges). Maps involve an additional dimension type, `dim_type.in_`.
We will only illustrate a simple `Map` object and some functionality here. `UnionMaps` are useful in the context of code generation and will be shown in the next section.

## Map

Think of maps as functions from some domain to some range. Maps can be applied to sets to produce new sets.
For example, consider the following map from the $ijk$-space to the 2D $ij$-plane defined. Think of this as the projection "down onto" the $ij$-plane.

In [84]:
# consider the same 3D cube domain
D = Set('[N]->{[i,j,k] : 0<=i,j,k<N}')

M = Map('[N]->{[i,j,k]->[i,j]}')

# apply the map M to the domain D
D1 = D.apply(M)

print('D: {}'.format(D))
print('D1: {}'.format(D1))

D: [N] -> { [i, j, k] : 0 <= i < N and 0 <= j < N and 0 <= k < N }
D1: [N] -> { [i0, i1] : 0 <= i0 < N and 0 <= i1 < N }


Notice that the index names have changed in the new set D1. This is because the output dimensions can be defined (generally) as any affine combination of the input dimensions.

For example, consider the map:

In [85]:
M1 = Map('[N]->{[i,j,k]->[2i+7j,17-i+3k]}')

D2 = D.apply(M1)

print('D: {}'.format(D))
print('D2: {}'.format(D2))

D: [N] -> { [i, j, k] : 0 <= i < N and 0 <= j < N and 0 <= k < N }
D2: [N] -> { [i0, i1] : exists (e0: -33 - N + i0 + 2i1 <= 6e0 <= -34 + i0 + 2i1 and -34 + i0 + 2i1 <= 7e0 <= -35 + N + i0 + 2i1 and -119 + 3i0 + 7i1 <= 21e0 <= -120 + N + 3i0 + 7i1) }


In general, the names of the output dimensions don't a nice interpretation relative to the inputs, so isl generates index names "i0", "i1", etc.

Also notice that the image of D by M1 here involves constraints with the "exists" clause. These correspond to "existentially quantifiers". Ctrl+f for "existential" in the isl manual linked above for more information. These constraints are represented by `dim_type.div` dimensions.

# Code generation

The isl library also supports code generation. This illustrates a simple example. Consider the following domain (same as above), defined by a `UnionSet`.

In [86]:
D = UnionSet('[N]->{A[i,j]: 0<=i,j<N; B[i,j,k] : 0<=i,j,k<N}')
M = UnionMap('''[N]->{
    A[i,j]->[i,j,0,0];
    B[i,j,k]->[i,j,1,k];
}''')

print(D)
print(M)

[N] -> { A[i, j] : 0 <= i < N and 0 <= j < N; B[i, j, k] : 0 <= i < N and 0 <= j < N and 0 <= k < N }
[N] -> { A[i, j] -> [i, j, 0, 0]; B[i, j, k] -> [i, j, 1, k] }
